In [1]:
import numpy as np 
import open3d as o3d
import time
import logging
import math
import trimesh

sample_directory = r"C:\Repo\SCAN2BIM-python\Samples\TrainingdataCreation"
voxel_size = 0.05

In [2]:
print("... FLOOR ...")

mesh_floor = o3d.io.read_triangle_mesh(r"C:\Data\Temp\012103-40-BIM-Gent\BIM\OBJ\floor.obj")
print(mesh_floor)
# o3d.visualization.draw_geometries([mesh_wall])
tri_mesh_floor = trimesh.Trimesh(vertices=np.asarray(mesh_floor.vertices), faces=np.asarray(mesh_floor.triangles))
# wall_triangles_centers = tri_mesh_wall.triangles_center
# tri_mesh_wall.show()


print("... WALL ...")

mesh_wall = o3d.io.read_triangle_mesh(r"C:\Data\Temp\012103-40-BIM-Gent\BIM\OBJ\wall.obj")
print(mesh_wall)
# o3d.visualization.draw_geometries([mesh_wall])
tri_mesh_wall = trimesh.Trimesh(vertices=np.asarray(mesh_wall.vertices), faces=np.asarray(mesh_wall.triangles))
# wall_triangles_centers = tri_mesh_wall.triangles_center
# tri_mesh_wall.show()


# print("... BEAM ...")
# mesh_beam = o3d.io.read_triangle_mesh(r"C:\Data\Temp\012103-40-BIM-Gent\BIM\OBJ\beam.obj")
# print(mesh_beam)
# tri_mesh_beam = trimesh.Trimesh(vertices=np.asarray(mesh_beam.vertices), faces=np.asarray(mesh_beam.triangles))
# beam_triangles_centers = tri_mesh_beam.triangles_center
# tri_mesh_beam.show()


meshes = [tri_mesh_floor, tri_mesh_wall] #, tri_mesh_beam]
mesh = trimesh.util.concatenate(meshes)
# mesh.show()
# # o3d.visualization.draw_geometries([meshpcd_beam])

# print("... COLUMN ...")
# mesh_column = o3d.io.read_triangle_mesh(r"C:\Data\Temp\012103-40-BIM-Gent\BIM\OBJ\column.obj")
# print(mesh_column)

# # o3d.visualization.draw_geometries([meshpcd_column])

... FLOOR ...
TriangleMesh with 3633 points and 1211 triangles.
... WALL ...
TriangleMesh with 9258 points and 3086 triangles.


In [3]:
# i=0

# total_length = len(tri_mesh_wall.triangles) + len(tri_mesh_beam.triangles)

# correct = 0
# wrong = 0

# while i < total_length:
#     while i < len(tri_mesh_wall.triangles):
#         if tri_mesh_wall.triangles[i].all() == mesh.triangles[i].all():
#             correct = correct + 1
#         else: 
#             wrong = wrong +1
#         i= i+1
#     while i < len(tri_mesh_wall.triangles) + len(tri_mesh_beam.triangles):
#         if tri_mesh_beam.triangles[i-len(tri_mesh_wall.triangles)].all() == mesh.triangles[i].all():
#             correct = correct + 1
#         else: 
#             wrong = wrong +1
#         i= i+1
# print("%s / %s " %(correct,total_length))

In [4]:
mesh_labels =[]
label_id = [ "Ceiling", "Floor", "Wall", "Beam", "Column", "Clutter"]
logging.info("LABELS:\n0: %s \n1: %s \n2: %s \n3: %s \n4: %s \n5: %s" % (label_id[0],label_id[1],label_id[2],label_id[3],label_id[4],label_id[5]))

length_floor = len(tri_mesh_floor.triangles)
i = 0
while i < length_floor:
    mesh_labels.append("1")
    i =  i+1

length_wall = len(tri_mesh_wall.triangles)
i = 0
while i < length_wall:
    mesh_labels.append("2")
    i =  i+1

# length_beam = len(tri_mesh_beam.triangles)
# i = 0
# while i < length_beam:
#     mesh_labels.append("3")
#     i =  i+1

print("%s / %s" %(len(mesh_labels), len(mesh.triangles)))


4297 / 4297


In [5]:
print("Load targeted pointcloudpoint cloud ...")
pointcloud_location = r"C:\Data\Temp\012103-40-BIM-Gent\RESULTS\Annotations\wall_part.pcd"
pcd = o3d.io.read_point_cloud(pointcloud_location)
print(pcd)

print("Downsampling pointcloud with voxel size: %s m" % voxel_size)
logging.info("Downsampling pointcloud with voxel size: %s m" % voxel_size)
pcd = pcd.voxel_down_sample(voxel_size)
print(pcd)
pcd.normalize_normals()
pcd.paint_uniform_color([1,1,1])

Load targeted pointcloudpoint cloud ...
PointCloud with 1375125 points.
Downsampling pointcloud with voxel size: 0.05 m
PointCloud with 92005 points.


PointCloud with 92005 points.

De Dichtste mesh vinden 

Normaal van het punt vergelijken met de normaal van de mesh

Afwijkende normale ditchen 


In [6]:
# Filter pointcloud on normals and assign labels.
i=0
Outlier_indeces = []
closest_faces = trimesh.proximity.nearby_faces(mesh, pcd.points)


In [7]:
print(len(closest_faces[5000]))

42


In [8]:
labels = ["5"]*len(pcd.points)
print(len(pcd.points))
print(len(labels))
print(len(closest_faces))
print("Clutter: %s" % labels.count("5"))
print("Wall: %s" % labels.count("2"))

92005
92005
92005
Clutter: 92005
Wall: 0


In [9]:
t = time.perf_counter()
i = 0
percentage = 0
To_far = 0
No_normal = 0
Match_neg = 0
Match_pos = 0
while i < len(pcd.points):
    Not_found = True

    point = np.asarray(pcd.points[i])
    # print("point: %s" % point)
    point_normal = np.asarray(pcd.normals[i])

    neirest_faces = closest_faces[i]
    # print("neirest_faces: %s" % neirest_faces)
    distances = []
    distances_id = []
    #Compute distance to neirest face
    for face in neirest_faces:
        # print("face: %s " %neirest_faces[i1])
        triangle = mesh.triangles[face]
        # print("triangle: %s" %triangle)
        closest_point = trimesh.triangles.closest_point([triangle],[pcd.points[i]])
        # print("closest point: %s " % closest_point)
        dist = np.linalg.norm(point - closest_point)
        # print("distance: %s" % dist)
        distances.append(dist)

    # print("... distances...")
    # print(distances)
    # print("Minimal distance: %s" % min(distances))
    
    #if min distance is lower than 0.15 cm then then a index list of distances is created from low to high
    if min(distances) > 0.15:
        np.asarray(pcd.colors)[i] = [0,0,0]
        To_far = To_far +1
        Not_found = False
    else:
        distances_id = np.argsort(distances)
    
    # print(distances)

    # print(" %s / %s / %s" %(len(neirest_faces),len(distances_id),len(distances)))


    i1 = 0

    
    while Not_found and i1 < len(distances_id) and distances[distances_id[i1]] <= 0.15:
        
        # print("index of distance: %s" % distances_id[i1])
        # print("index of triangle: %s" % neirest_faces[distances_id[i1]])
        triangle = mesh.triangles[neirest_faces[distances_id[i1]]]
        # print("triangle: %s" %triangle)
        mesh_normal = np.asarray(trimesh.triangles.normals(triangle))[0][0]
        # print("mesh normaal: %s " %mesh_normal)
        # print("mesh normaal norm: %s "% np.linalg.norm(mesh_normal))
        # print("point normaal: %s " %point_normal)
        # print("point normaal norm: %s "% np.linalg.norm(point_normal))

        # print("Dot product: %s " % np.dot(np.asarray(point_normal), np.asarray(mesh_normal)))

        if np.dot(np.asarray(point_normal), np.asarray(mesh_normal)) > 0.6:
            labels[i] = mesh_labels[neirest_faces[distances_id[i1]]]
            np.asarray(pcd.colors)[i] = [0,0,1]
            Match_pos = Match_pos +1
            Not_found = False
        if np.dot(np.asarray(point_normal), np.asarray(mesh_normal)) < -0.6:
            labels[i] = mesh_labels[neirest_faces[distances_id[i1]]]
            np.asarray(pcd.colors)[i] = [0,1,1]
            Match_neg = Match_neg +1
        i1  = i1+1
    if Not_found and pcd.colors[i].all() == np.asarray([1,1,1]).all():
        np.asarray(pcd.colors)[i] = [1,0,0]
        # print("No matching normal")
        No_normal = No_normal +1 

    progress = i/len(np.asarray(pcd.points))*100
    if progress >= percentage:
        timing = time.perf_counter()
        dtiming = timing - t
        hours = math.floor(dtiming/3600)
        minutes = math.floor(dtiming/60 - hours*60)
        seconds = dtiming - hours*3600 - minutes*60
        print("progress: %s %% - %sh %smin %ss" %(round(progress),hours, minutes, seconds))
        percentage = percentage + 5
    i= i+1
print("Points to far: %s//%s" %(To_far, len(pcd.points)))
print("Points with pos matching normal: %s/%s" %(Match_pos, len(pcd.points)))
print("Points with neg matching normal: %s/%s" %(Match_neg, len(pcd.points)))
print("Points with no matching normal: %s/%s" %(No_normal, len(pcd.points)))

progress: 0 % - 0h 0min 0.008000599999832048s


C:\Users\SamDeGeyter\anaconda3\envs\scan2bim\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


progress: 5 % - 0h 0min 17.80085289999988s
progress: 10 % - 0h 0min 36.21082209999986s
progress: 15 % - 0h 0min 55.00351010000031s
progress: 20 % - 0h 1min 12.458414800000355s
progress: 25 % - 0h 1min 30.720494399999552s
progress: 30 % - 0h 1min 52.8060945999996s
progress: 35 % - 0h 2min 12.929670299999998s
progress: 40 % - 0h 2min 34.884430000000066s
progress: 45 % - 0h 2min 58.701539100000446s
progress: 50 % - 0h 3min 20.97893060000024s
progress: 55 % - 0h 3min 47.66340610000043s
progress: 60 % - 0h 4min 9.915358500000366s
progress: 65 % - 0h 4min 30.041446199999882s
progress: 70 % - 0h 4min 46.0179177s
progress: 75 % - 0h 5min 3.7186099000000468s
progress: 80 % - 0h 5min 23.071425899999667s
progress: 85 % - 0h 5min 43.38185760000033s
progress: 90 % - 0h 6min 5.500499300000229s
progress: 95 % - 0h 6min 21.143889500000114s
Points to far: 29881//92005
Points with pos matching normal: 31902/92005
Points with neg matching normal: 27339/92005
Points with no matching normal: 15885/92005


In [10]:
t = time.perf_counter()
i=0
percentage = 0


while i < len(np.asarray(pcd.points)):
    if labels[i] == "0":
        np.asarray(pcd.colors)[i] = [0,0,0]
 

    elif labels[i] == "1":
        np.asarray(pcd.colors)[i] = [1,0,0]
       

    elif labels[i] == "2":
        np.asarray(pcd.colors)[i] = [0,1,0]
   

    elif labels[i] == "3":
        np.asarray(pcd.colors)[i] = [0,0,1]

    elif labels[i] == "4":
        np.asarray(pcd.colors)[i] = [1,1,0]
    elif labels[i] == "5":
        np.asarray(pcd.colors)[i] = [1,1,1]
      

    progress = i/len(np.asarray(pcd.points))*100
    if progress >= percentage:
        timing = time.perf_counter()
        dtiming = timing - t
        hours = math.floor(dtiming/3600)
        minutes = math.floor(dtiming/60 - hours*60)
        seconds = dtiming - hours*3600 - minutes*60
        print("progress: %s %% - %sh %smin %ss" %(round(progress),hours, minutes, seconds))
        percentage = percentage + 5
    i= i+1

print("...DONE...")


progress: 0 % - 0h 0min 0.00023090000013326062s
progress: 5 % - 0h 0min 0.04415340000014112s
progress: 10 % - 0h 0min 0.08778940000047442s
progress: 15 % - 0h 0min 0.1308853000000454s
progress: 20 % - 0h 0min 0.17338509999990492s
progress: 25 % - 0h 0min 0.21567640000012034s
progress: 30 % - 0h 0min 0.2580625999999029s
progress: 35 % - 0h 0min 0.3009984999998778s
progress: 40 % - 0h 0min 0.342952800000603s
progress: 45 % - 0h 0min 0.3851168000001053s
progress: 50 % - 0h 0min 0.4274875000000975s
progress: 55 % - 0h 0min 0.46985490000042773s
progress: 60 % - 0h 0min 0.5117461999998341s
progress: 65 % - 0h 0min 0.5548110999998244s
progress: 70 % - 0h 0min 0.6022804000003816s
progress: 75 % - 0h 0min 0.6448311000003741s
progress: 80 % - 0h 0min 0.6868480000002819s
progress: 85 % - 0h 0min 0.729542400000355s
progress: 90 % - 0h 0min 0.7747527000001355s
progress: 95 % - 0h 0min 0.8169576000000234s
...DONE...


In [ ]:
o3d.visualization.draw_geometries([pcd])